In [0]:
storage_account_name = "health1care1gen2stg"
container_name = "gold"
access_key = "DQsMJ8waN/6oonXhT8JihU00r+aLJ7BDDF2kGffskG+r0GBqNsH7vSHsFc31hkRSZNdDsn03zIzf+AStGUeVoQ==" 

spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    access_key
)

In [0]:
%sql
use health_care_hive_met;

## 1. Which disease has a maximum number of claims.

In [0]:
%sql
SELECT 
    disease_name,
    COUNT(claim_id) AS claim_count
FROM 
    health_care_hive_met.hlt_slv_claims_managed
GROUP BY 
    disease_name
ORDER BY 
    claim_count DESC
LIMIT 1;

disease_name,claim_count
Pet allergy,3


In [0]:
topic_name = "001_disease_maximum_claims"
# First, run the SQL query and store the result in a DataFrame
dis_max_clm_df = spark.sql("""
    SELECT 
    disease_name,
    COUNT(claim_id) AS claim_count
FROM 
    health_care_hive_met.hlt_slv_claims_managed
GROUP BY 
    disease_name
ORDER BY 
    claim_count DESC
LIMIT 1
""")
display(dis_max_clm_df)

# Define the path in ADLS Gen 2 where you want to store the CSV file
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{topic_name}/"

# Write the DataFrame to the specified ADLS Gen 2 path as a CSV file
dis_max_clm_df.write.parquet(output_path, mode='overwrite', compression='snappy')


disease_name,claim_count
Pet allergy,3


## 2. Find those Subscribers having age less than 30 and they subscribe any subgroup

In [0]:
%sql
SELECT 
    subscriber_id,
    first_name,
    last_name,
    birth_date,
    subgrp_id
FROM 
    hlt_slv_subscriber_managed
WHERE 
    subgrp_id IS NOT NULL
    AND DATEDIFF(current_date(), birth_date) / 365 < 30;

subscriber_id,first_name,last_name,birth_date,subgrp_id
SUBID10017,Bandhu,Seth,1996-10-15,S108
SUBID10083,Bhilangana,Pandit,1995-01-04,S109
SUBID10093,Chandavarman,Singh,1997-05-10,S110


In [0]:
topic_name = "002_sub_with_age_less_30"
# First, run the SQL query and store the result in a DataFrame
subscribers_df = spark.sql("""
    SELECT 
        subscriber_id,
        first_name,
        last_name,
        birth_date,
        subgrp_id
    FROM 
        hlt_slv_subscriber_managed
    WHERE 
        subgrp_id IS NOT NULL
        AND DATEDIFF(current_date(), birth_date) / 365 < 30
""")
display(subscribers_df)

# Define the path in ADLS Gen 2 where you want to store the CSV file
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{topic_name}/"

# Write the DataFrame to the specified ADLS Gen 2 path as a CSV file
subscribers_df.write.parquet(output_path, mode='overwrite', compression='snappy')

subscriber_id,first_name,last_name,birth_date,subgrp_id
SUBID10017,Bandhu,Seth,1996-10-15,S108
SUBID10083,Bhilangana,Pandit,1995-01-04,S109
SUBID10093,Chandavarman,Singh,1997-05-10,S110


## 3. Find out which group has maximum subgroups.

In [0]:
%sql
SELECT 
    g.group_id,
    g.group_name,
    COUNT(s.subgrp_id) AS subgroup_count
FROM 
    hlt_slv_group_managed g
JOIN 
    hlt_slv_grpsubgrp_managed gs ON g.group_id = gs.group_id
JOIN 
    hlt_slv_subgroup_managed s ON gs.sub_group_id = s.subgrp_id
GROUP BY 
    g.group_id, g.group_name
ORDER BY 
    subgroup_count DESC
LIMIT 1;

group_id,group_name,subgroup_count
GRP147,Raheja QBE General Insurance,2


In [0]:
topic_name = "003_grp_max_subgrps"
# First, run the SQL query and store the result in a DataFrame
grp_max_subgrps_df = spark.sql("""
    SELECT 
    g.group_id,
    g.group_name,
    COUNT(s.subgrp_id) AS subgroup_count
FROM 
    hlt_slv_group_managed g
JOIN 
    hlt_slv_grpsubgrp_managed gs ON g.group_id = gs.group_id
JOIN 
    hlt_slv_subgroup_managed s ON gs.sub_group_id = s.subgrp_id
GROUP BY 
    g.group_id, g.group_name
ORDER BY 
    subgroup_count DESC
LIMIT 1
""")
display(grp_max_subgrps_df)

# Define the path in ADLS Gen 2 where you want to store the CSV file
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{topic_name}/"

# Write the DataFrame to the specified ADLS Gen 2 path as a CSV file
grp_max_subgrps_df.write.parquet(output_path, mode='overwrite', compression='snappy')

group_id,group_name,subgroup_count
GRP147,Raheja QBE General Insurance,2


## 4. Find out the hospital that serves the most number of patients

In [0]:
%sql
SELECT 
    h.hospital_id,
    h.hospital_name,
    COUNT(p.patient_id) AS patient_count
FROM 
    hlt_slv_hospital_managed h
JOIN 
    hlt_slv_patient_managed p ON h.hospital_id = p.hospital_id
GROUP BY 
    h.hospital_id, h.hospital_name
ORDER BY 
    patient_count DESC
LIMIT 1;


hospital_id,hospital_name,patient_count
H1017,Manipal Hospitals,9


In [0]:
topic_name = "004_hosptl_most_num_of_patients"
# First, run the SQL query and store the result in a DataFrame
grp_max_subgrps_df = spark.sql("""
   SELECT 
    h.hospital_id,
    h.hospital_name,
    COUNT(p.patient_id) AS patient_count
FROM 
    hlt_slv_hospital_managed h
JOIN 
    hlt_slv_patient_managed p ON h.hospital_id = p.hospital_id
GROUP BY 
    h.hospital_id, h.hospital_name
ORDER BY 
    patient_count DESC
LIMIT 1
""")
display(grp_max_subgrps_df)

# Define the path in ADLS Gen 2 where you want to store the CSV file
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{topic_name}/"

# Write the DataFrame to the specified ADLS Gen 2 path as a CSV file
grp_max_subgrps_df.write.parquet(output_path, mode='overwrite', compression='snappy')

hospital_id,hospital_name,patient_count
H1017,Manipal Hospitals,9


## 5. Find out which subgroups are subscribed to the most number of times

In [0]:
%sql
SELECT 
    s.subgrp_id,
    s.sub_grp_name,
    COUNT(sm.subscriber_id) AS subscriber_count
FROM 
    hlt_slv_subgroup_managed s
JOIN 
    hlt_slv_subscriber_managed sm ON s.subgrp_id = sm.subgrp_id
GROUP BY 
    s.subgrp_id, s.sub_grp_name
ORDER BY 
    subscriber_count DESC
LIMIT 1;


subgrp_id,sub_grp_name,subscriber_count
S104,Therapy,13


In [0]:
topic_name = "005_subgrps_subs_most_num_of_times"
# First, run the SQL query and store the result in a DataFrame
subgrp_max_subs_df = spark.sql("""
   SELECT 
    s.subgrp_id,
    s.sub_grp_name,
    COUNT(sm.subscriber_id) AS subscriber_count
FROM 
    hlt_slv_subgroup_managed s
JOIN 
    hlt_slv_subscriber_managed sm ON s.subgrp_id = sm.subgrp_id
GROUP BY 
    s.subgrp_id, s.sub_grp_name
ORDER BY 
    subscriber_count DESC
LIMIT 1
""")
display(subgrp_max_subs_df)

# Define the path in ADLS Gen 2 where you want to store the CSV file
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{topic_name}/"

# Write the DataFrame to the specified ADLS Gen 2 path as a CSV file
subgrp_max_subs_df.write.parquet(output_path, mode='overwrite', compression='snappy')

subgrp_id,sub_grp_name,subscriber_count
S104,Therapy,13


## 6. Find out the total number of claims that were rejected

In [0]:
%sql
SELECT 
    COUNT(*) AS rejected_claims_count
FROM 
    hlt_slv_claims_managed
WHERE 
    claim_accept_in = 'Rejected';

rejected_claims_count
0


In [0]:
topic_name = "006_total_claims_rejected"
# First, run the SQL query and store the result in a DataFrame
tol_claims_rej_df = spark.sql("""
   SELECT 
    COUNT(*) AS rejected_claims_count
FROM 
    hlt_slv_claims_managed
WHERE 
    claim_accept_in = 'Rejected'
""")
display(tol_claims_rej_df)

# Define the path in ADLS Gen 2 where you want to store the CSV file
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{topic_name}/"

# Write the DataFrame to the specified ADLS Gen 2 path as a CSV file
tol_claims_rej_df.write.parquet(output_path, mode='overwrite', compression='snappy')

rejected_claims_count
0


## 7. Find out from which city most claims are coming

In [0]:
%sql
SELECT 
    p.city,
    COUNT(c.claim_id) AS claim_count
FROM 
    hlt_slv_claims_managed c
JOIN 
    hlt_slv_patient_managed p ON c.patient_id = p.patient_id
GROUP BY 
    p.city
ORDER BY 
    claim_count DESC
LIMIT 1;

city,claim_count
Mysore,2


In [0]:
topic_name = "007_city_most_claims"
# First, run the SQL query and store the result in a DataFrame
city_most_claims_df = spark.sql("""
   SELECT 
    p.city,
    COUNT(c.claim_id) AS claim_count
FROM 
    hlt_slv_claims_managed c
JOIN 
    hlt_slv_patient_managed p ON c.patient_id = p.patient_id
GROUP BY 
    p.city
ORDER BY 
    claim_count DESC
LIMIT 1
""")
display(city_most_claims_df)

# Define the path in ADLS Gen 2 where you want to store the CSV file
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{topic_name}/"

# Write the DataFrame to the specified ADLS Gen 2 path as a CSV file
city_most_claims_df.write.parquet(output_path, mode='overwrite', compression='snappy')

city,claim_count
Mysore,2


## 8. Find out which groups of policies subscribers subscribe to mostly (Government or Private)

In [0]:
%sql
SELECT 
    g.group_type,
    COUNT(sm.subscriber_id) AS subscriber_count
FROM 
    hlt_slv_group_managed g
JOIN 
    hlt_slv_grpsubgrp_managed gs ON g.group_id = gs.group_id
JOIN 
    hlt_slv_subscriber_managed sm ON gs.sub_group_id = sm.subgrp_id
GROUP BY 
    g.group_type
ORDER BY 
    subscriber_count DESC
LIMIT 1;


group_type,subscriber_count
Private,338


In [0]:
topic_name = "008_grps_of_policies"
# First, run the SQL query and store the result in a DataFrame
grps_of_policies_df = spark.sql("""
   SELECT 
    p.city,
    COUNT(c.claim_id) AS claim_count
FROM 
    hlt_slv_claims_managed c
JOIN 
    hlt_slv_patient_managed p ON c.patient_id = p.patient_id
GROUP BY 
    p.city
ORDER BY 
    claim_count DESC
LIMIT 1
""")
display(grps_of_policies_df)

# Define the path in ADLS Gen 2 where you want to store the CSV file
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{topic_name}/"

# Write the DataFrame to the specified ADLS Gen 2 path as a CSV file
grps_of_policies_df.write.parquet(output_path, mode='overwrite', compression='snappy')

city,claim_count
Mysore,2


## 9. Calculate the average monthly premium paid by subscribers

In [0]:
%sql
SELECT 
    AVG(sg.monthly_premium) AS average_monthly_premium
FROM 
    hlt_slv_subgroup_managed sg
JOIN 
    hlt_slv_subscriber_managed sm ON sg.subgrp_id = sm.subgrp_id;


average_monthly_premium
1867.3469387755101


In [0]:
topic_name = "009_avg_monthly_prem"
# First, run the SQL query and store the result in a DataFrame
avg_monthly_prem_df = spark.sql("""
   SELECT 
    AVG(sg.monthly_premium) AS average_monthly_premium
FROM 
    hlt_slv_subgroup_managed sg
JOIN 
    hlt_slv_subscriber_managed sm ON sg.subgrp_id = sm.subgrp_id
""")
display(avg_monthly_prem_df)

# Define the path in ADLS Gen 2 where you want to store the CSV file
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{topic_name}/"

# Write the DataFrame to the specified ADLS Gen 2 path as a CSV file
avg_monthly_prem_df.write.parquet(output_path, mode='overwrite', compression='snappy')

average_monthly_premium
1867.3469387755101


## 10. Find out which group is the most profitable

In [0]:
%sql
SELECT 
    g.group_id,
    g.group_name,
    SUM(g.premium_written) AS total_premium
FROM 
    hlt_slv_group_managed g
GROUP BY 
    g.group_id, g.group_name
ORDER BY 
    total_premium DESC
LIMIT 1;


group_id,group_name,total_premium
GRP123,IndiaFirst Life Insurance Co. Ltd.,99000


In [0]:
topic_name = "010_grp_most_profit"
# First, run the SQL query and store the result in a DataFrame
grp_most_profit_df = spark.sql("""
   SELECT 
    g.group_id,
    g.group_name,
    SUM(g.premium_written) AS total_premium
FROM 
    hlt_slv_group_managed g
GROUP BY 
    g.group_id, g.group_name
ORDER BY 
    total_premium DESC
LIMIT 1
""")
display(grp_most_profit_df)

# Define the path in ADLS Gen 2 where you want to store the CSV file
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{topic_name}/"

# Write the DataFrame to the specified ADLS Gen 2 path as a CSV file
grp_most_profit_df.write.parquet(output_path, mode='overwrite', compression='snappy')

group_id,group_name,total_premium
GRP123,IndiaFirst Life Insurance Co. Ltd.,99000


## 11. List all patients below who were admitted for cancer

In [0]:
%sql
SELECT 
    p.patient_id,
    p.patient_name,
    p.patient_birth_date,
    p.disease_name,
    p.hospital_id
FROM 
    hlt_slv_patient_managed p
WHERE 
    p.disease_name LIKE '%cancer%'
    AND p.Patient_name IS NOT NULL;

patient_id,patient_name,patient_birth_date,disease_name,hospital_id
112766,Brahmdev,1948-12-20,Bladder cancer,H1016
199252,Ujjawal,1980-04-16,Kidney cancer,H1009
156988,Bhagavaana,1935-09-16,Breast cancer,H1012
121783,Paridhi,1959-03-27,Bladder cancer,H1013
197441,Deependu,1952-02-13,Lung cancer,H1001


In [0]:
topic_name = "011_patient_with_cancer"
# First, run the SQL query and store the result in a DataFrame
patient_with_cancer_df = spark.sql("""
   SELECT 
    p.patient_id,
    p.patient_name,
    p.patient_birth_date,
    p.disease_name,
    p.hospital_id
FROM 
    hlt_slv_patient_managed p
WHERE 
    p.disease_name LIKE '%cancer%'
    AND p.Patient_name IS NOT NULL
""")
display(patient_with_cancer_df)

# Define the path in ADLS Gen 2 where you want to store the CSV file
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{topic_name}/"

# Write the DataFrame to the specified ADLS Gen 2 path as a CSV file
patient_with_cancer_df.write.parquet(output_path, mode='overwrite', compression='snappy')

patient_id,patient_name,patient_birth_date,disease_name,hospital_id
112766,Brahmdev,1948-12-20,Bladder cancer,H1016
199252,Ujjawal,1980-04-16,Kidney cancer,H1009
156988,Bhagavaana,1935-09-16,Breast cancer,H1012
121783,Paridhi,1959-03-27,Bladder cancer,H1013
197441,Deependu,1952-02-13,Lung cancer,H1001


## 12. List patients who have cashless insurance and have total charges greater than or equal to Rs. 50,000

In [0]:
%sql
SELECT 
    p.patient_id,
    p.patient_name,
    c.claim_amount
FROM 
    hlt_slv_claims_managed c
JOIN 
    hlt_slv_patient_managed p ON c.patient_id = p.patient_id
WHERE 
    c.claim_type = 'claims of policy'
    AND c.claim_amount >= 50000
    AND p.Patient_name IS NOT NULL;

patient_id,patient_name,claim_amount
112766,Brahmdev,151142.0
171320,Atasi,64840.0
184479,Bandhu,161786.0
156434,Pushti,122806.0
138861,Upasana,62985.0
148674,Ayushmati,173600.0
133107,Drashti,160739.0
197441,Deependu,156557.0
110690,Laksman,130339.0
167423,Chittesh,161497.0


In [0]:
topic_name = "012_cashless_ins_claim"
# First, run the SQL query and store the result in a DataFrame
cashless_ins_claim_df = spark.sql("""
SELECT 
    p.patient_id,
    p.patient_name,
    c.claim_amount
FROM 
    hlt_slv_claims_managed c
JOIN 
    hlt_slv_patient_managed p ON c.patient_id = p.patient_id
WHERE 
    c.claim_type = 'claims of policy'
    AND c.claim_amount >= 50000
    AND p.Patient_name IS NOT NULL
""")
display(cashless_ins_claim_df)

# Define the path in ADLS Gen 2 where you want to store the CSV file
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{topic_name}/"

# Write the DataFrame to the specified ADLS Gen 2 path as a CSV file
cashless_ins_claim_df.write.parquet(output_path, mode='overwrite', compression='snappy')

patient_id,patient_name,claim_amount
112766,Brahmdev,151142.0
171320,Atasi,64840.0
184479,Bandhu,161786.0
156434,Pushti,122806.0
138861,Upasana,62985.0
148674,Ayushmati,173600.0
133107,Drashti,160739.0
197441,Deependu,156557.0
110690,Laksman,130339.0
167423,Chittesh,161497.0


## 13. List female patients have allergy

In [0]:
%sql
SELECT 
    p.patient_id,
    p.patient_name,
    p.patient_gender,
    p.patient_birth_date,
    p.disease_name
FROM 
    hlt_slv_patient_managed p
WHERE 
    p.patient_gender = 'Female'
    AND DATEDIFF(current_date(), p.patient_birth_date) / 365 > 40
    AND p.disease_name LIKE '%allergy%'
    AND p.Patient_name IS NOT NULL;

patient_id,patient_name,patient_gender,patient_birth_date,disease_name
172579,Devnath,Female,1946-05-01,Food allergy
148137,Umang,Female,1963-07-14,Pet allergy
119268,Shivakari,Female,1944-11-02,Mold allergy


In [0]:
topic_name = "013_female_patients_allergy"
# First, run the SQL query and store the result in a DataFrame
female_allergy_df = spark.sql("""
SELECT 
    p.patient_id,
    p.patient_name,
    p.patient_gender,
    p.patient_birth_date,
    p.disease_name
FROM 
    hlt_slv_patient_managed p
WHERE 
    p.patient_gender = 'Female'
    AND DATEDIFF(current_date(), p.patient_birth_date) / 365 > 40
    AND p.disease_name LIKE '%allergy%'
    AND p.Patient_name IS NOT NULL
""")
display(female_allergy_df)

# Define the path in ADLS Gen 2 where you want to store the CSV file
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{topic_name}/"

# Write the DataFrame to the specified ADLS Gen 2 path as a CSV file
female_allergy_df.write.parquet(output_path, mode='overwrite', compression='snappy')

patient_id,patient_name,patient_gender,patient_birth_date,disease_name
172579,Devnath,Female,1946-05-01,Food allergy
148137,Umang,Female,1963-07-14,Pet allergy
119268,Shivakari,Female,1944-11-02,Mold allergy
